In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [17]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
vocab_size = len(chars)

232307


In [5]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [14]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [22]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('input')
print(x)
print('target')
print(y)

input
tensor([[72,  1, 54, 67, 57,  0, 59, 65],
        [58, 71,  1, 54, 67, 57,  1, 66],
        [64, 67, 62, 75, 58, 72,  9,  1],
        [57,  9,  3,  1, 61, 58,  1, 58]], device='cuda:0')
target
tensor([[ 1, 54, 67, 57,  0, 59, 65, 74],
        [71,  1, 54, 67, 57,  1, 66, 68],
        [67, 62, 75, 58, 72,  9,  1, 76],
        [ 9,  3,  1, 61, 58,  1, 58, 77]], device='cuda:0')


In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(28)
when input is tensor([80, 28]) target is tensor(39)
when input is tensor([80, 28, 39]) target is tensor(42)
when input is tensor([80, 28, 39, 42]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [18]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      index_next = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, index_next), dim=1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


﻿N4v!ziM[CY]qgin_24jaxcSb_n?iarYs*64HPv1tiMSeX 6hg0CW3sqg0;_8&IDmFr_KrwA5sfrkUj!;Zp6OQvZLD*MGrFo"i2'S_I*2)pF*o"F[uKv3 bv3L5x '4kAkA0]CiC U8VK8?q'(XyrT)N"iuRJ3IX7Q2zR;yTReLDaSlPW9&IP7z'7'p*Khi2B0&_60yiC Oi5Yocj]I H;-b[yyTzheY-TGBO-u zC vMp*7Fp*75]XM;1'6BjVFnbed5Cp*gfwDN"c
qAC -ulG](XUD﻿0f7.um2t;(XQpaMJm&;:UVKrR( _]?BvO2CwT)P(XQNpwOHuW;-EkJ8?Kl;[Kt3ftCbIX),0]DWrqgrel0HIwLL9D9k O*c7Qw0Xq
T)Y]OHEb!bC﻿Xt_AMsy4j_DrH9iO;D8G﻿X7B,.*)jI
'Sd"dQl2t﻿P(Z'?.ja!JLTZ1e-H*Bfzl[_C .hSdY]Ks3m15o:O9-abIKSuX]Uf*SKnNm


In [20]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [23]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.693, val loss: 4.688
step: 250, train loss: 4.641, val loss: 4.628
step: 500, train loss: 4.572, val loss: 4.573
step: 750, train loss: 4.520, val loss: 4.501
4.428039073944092


In [24]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


g0JMtZAMOT!TpF'w!01'MGisH_ftxPr l5SGbn'SGz8f7"OimkzdGL60Ws
UeXfwLfcZ:gKC F(XtE]t﻿X.j,.zqql-)1!qY0O*Ayf4g il05nDk:d*0f9"﻿JBkuJ3pIXkiNT1AJ
6I?WFsm1Ajw.K:4SNN"2'VDV:eLEd-hO2CRUe;xv)11N"eX3ya.dvL.GU;KrLt tXaM]t_B2 pa.fV_sP-ve5]*Ek!]KN"p5][(M:OF.Ir﻿v2[?;t8rep:ed .zi5SB(djy.SHY8R2aDc"O46;dGo3f1QwO7.JcvlM:j,c2]t9k﻿.8CR O4]?xkCv&b.QbAhaczdQZWeY-wz'6M﻿_RHMLyV!Qh0&td2lFDinIlOHP1j&'﻿.zor,(iF,V9T)sSovlix l)4jy3Lis_A-TX?H)e][isvix9rb,?6:9o" 5R6"i
Wn(BDg
SqeXL.6xlxZ1o"T:w!Z41TB-u-H:Xyt11'6&H[C l4kwBR;m?Eul&Sl


---